In [2]:
library(tidyverse)
library(tidymodels)
library(lubridate)
library(ggplot2)
library(cowplot)

players <- read_csv("data/players.csv")
sessions <- read_csv("data/sessions.csv")
players <- players %>%
  mutate(hashedEmail = str_sub(hashedEmail, 1, 10))
sessions <- sessions %>%
  mutate(hashedEmail = str_sub(hashedEmail, 1, 10))
head(players)
head(sessions)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0,0.0,Blake,Male,17
Amateur,TRUE,23fe711e0e,0.7,Flora,Female,21
Regular,TRUE,7dc01f10bf,0.1,Kylie,Male,21
Amateur,TRUE,f58aad5996,0.0,Adrian,Female,17


hashedEmail,start_time,end_time,original_start_time,original_end_time
<chr>,<chr>,<chr>,<dbl>,<dbl>
bfce39c89d,30/06/2024 18:12,30/06/2024 18:24,1.71977e+12,1.71977e+12
36d9cbb4c6,17/06/2024 23:33,17/06/2024 23:46,1.71867e+12,1.71867e+12
f8f5477f5a,25/07/2024 17:34,25/07/2024 17:57,1.72193e+12,1.72193e+12
bfce39c89d,25/07/2024 03:22,25/07/2024 03:58,1.72188e+12,1.72188e+12
36d9cbb4c6,25/05/2024 16:01,25/05/2024 16:12,1.71665e+12,1.71665e+12
bfce39c89d,23/06/2024 15:08,23/06/2024 17:10,1.71916e+12,1.71916e+12


In [3]:
sessions <- sessions %>%
  mutate(
    start_dt = dmy_hm(start_time),
    end_dt   = dmy_hm(end_time),
    duration_mins = as.numeric(difftime(end_dt, start_dt, units = "mins")),
    hour = hour(start_dt),
    wday = wday(start_dt, label = TRUE),
    night = if_else(hour >= 20 | hour < 6, 1, 0),
    weekend = if_else(wday %in% c("Sat", "Sun"), 1, 0)
  )

session_features <- sessions %>%
  group_by(hashedEmail) %>%
  summarize(
    n_sessions = n(),
    avg_duration = mean(duration_mins, na.rm = TRUE),
    prop_night = mean(night, na.rm = TRUE),
    prop_weekend = mean(weekend, na.rm = TRUE),
    .groups = "drop"
  )
session_features <- session_features %>%
  mutate(hashedEmail = str_sub(hashedEmail, 1, 10))
head(session_features)

hashedEmail,n_sessions,avg_duration,prop_night,prop_weekend
<chr>,<int>,<dbl>,<dbl>,<dbl>
0088b5e134,2,53.00000,1.0000000,0.0000000
060aca80f8,1,30.00000,1.0000000,0.0000000
0ce7bfa910,1,11.00000,1.0000000,0.0000000
0d4d71be33,13,32.15385,0.6923077,0.4615385
0d70dd9cac,2,35.00000,0.5000000,0.5000000
11006065e9,1,10.00000,0.0000000,1.0000000


In [4]:
modeling_data <- players %>%
  left_join(session_features, by = "hashedEmail") %>%
  replace_na(list(n_sessions = 0, avg_duration = 0, 
                  prop_night = 0, prop_weekend = 0)) %>%
  mutate(subscribe = if_else(subscribe, "Subscribed", "Not Subscribed") %>% 
                     factor(levels = c("Not Subscribed", "Subscribed")),
         experience = factor(experience, 
                             levels = c("Beginner", "Amateur", "Regular", 
                                        "Veteran", "Pro"),
                             ordered = FALSE)) %>%
  select(-hashedEmail, -name)

glimpse(modeling_data)

Rows: 196
Columns: 9
$ experience   <fct> Pro, Veteran, Veteran, Amateur, Regular, Amateur, Regular…
$ subscribe    <fct> Subscribed, Subscribed, Not Subscribed, Subscribed, Subsc…
$ played_hours <dbl> 30.3, 3.8, 0.0, 0.7, 0.1, 0.0, 0.0, 0.0, 0.1, 0.0, 1.6, 0…
$ gender       <chr> "Male", "Male", "Male", "Female", "Male", "Female", "Fema…
$ Age          <dbl> 9, 17, 17, 21, 21, 17, 19, 21, 17, 22, 23, 17, 25, 22, 17…
$ n_sessions   <int> 27, 3, 1, 1, 1, 0, 0, 1, 1, 0, 3, 0, 1, 1, 0, 0, 0, 37, 2…
$ avg_duration <dbl> 74.77778, 85.00000, 5.00000, 50.00000, 9.00000, 0.00000, …
$ prop_night   <dbl> 0.8518519, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 0.…
$ prop_weekend <dbl> 0.5555556, 0.0000000, 0.0000000, 1.0000000, 0.0000000, 0.…


In [5]:
set.seed(123)
data_split <- initial_split(modeling_data, prop = 0.75, strata = subscribe)
train_data <- training(data_split)
test_data  <- testing(data_split)

In [6]:
subscription_recipe <- recipe(subscribe ~ experience + played_hours + n_sessions + 
                                avg_duration + prop_night + prop_weekend, 
                              data = train_data)%>%
  step_mutate(subscribe = factor(subscribe, levels = c("Not Subscribed", "Subscribed")))

logistic_spec <- logistic_reg() %>%
  set_engine("glm") %>%
  set_mode("classification")

subscription_workflow <- workflow() %>%
  add_recipe(subscription_recipe) %>%
  add_model(logistic_spec)

subscription_recipe
logistic_spec



── Recipe ──────────────────────────────────────────────────────────────────────



── Inputs 

Number of variables by role

outcome:   1
predictor: 6



── Operations 

• Variable mutation for: factor(subscribe, levels = c("Not Subscribed",
  "Subscribed"))



Logistic Regression Model Specification (classification)

Computational engine: glm 


In [7]:
set.seed(123)
cv_folds <- vfold_cv(train_data, v = 5, strata = subscribe)

cv_results <- fit_resamples(
  subscription_workflow,
  resamples = cv_folds,
  metrics = metric_set(accuracy, roc_auc),
  control = control_resamples(save_pred = TRUE)
)

cv_metrics <- collect_metrics(cv_results)
cv_metrics

→ A | warning: glm.fit: fitted probabilities numerically 0 or 1 occurred

There were issues with some computations   A: x1

There were issues with some computations   A: x5





.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,binary,0.7210181,5,0.006325144,Preprocessor1_Model1
roc_auc,binary,0.4641543,5,0.021288113,Preprocessor1_Model1


In [8]:
final_results <- last_fit(subscription_workflow, split = data_split, 
                          metrics = metric_set(accuracy, roc_auc))

test_metrics <- collect_metrics(final_results)
test_metrics

→ A | warning: glm.fit: fitted probabilities numerically 0 or 1 occurred

There were issues with some computations   A: x1

There were issues with some computations   A: x1





.metric,.estimator,.estimate,.config
<chr>,<chr>,<dbl>,<chr>
accuracy,binary,0.7346939,Preprocessor1_Model1
roc_auc,binary,0.4433761,Preprocessor1_Model1
